In [1]:
import pandas as pd
from collections import Counter
import torch
import torch.nn.functional as F

In [2]:
CUTOFF = 50

In [3]:
df = pd.read_csv('data/parsed.csv')

In [5]:
letters = set()
for i in df['text']:
    for j in i:
        letters.add(j) 

In [7]:
all_text = ''
for i in df['text']:
    all_text += i.lower()
counter = pd.Series(Counter(all_text)).sort_values()

In [8]:
import string
alp='абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
counter_rus = counter[counter.index.to_series().isin(list(alp))]

In [11]:
counter = counter[counter > CUTOFF]

In [12]:
counter_list = counter.index.to_list()

In [25]:
class Tokenizator:
    def __init__(self, tokens: list, max_length: int):
        self.num_tokens = len(tokens)
        self.tokens = tokens
        self.max_length = max_length
        
    def tokenize(self, text: str) -> torch.Tensor:
        res_text = []
        
        if len(text) > self.max_length:
            text = text[:self.max_length]
        
        for i in text:
            if i not in self.tokens:
                continue
            res_letter = [0]*self.num_tokens
            res_letter[self.tokens.index(i)] = 1
            res_text.append(res_letter)
            
        difference = self.max_length - len(res_text)
        for i in range(difference):
            res_text.append([0]*self.num_tokens)
            
        return torch.Tensor(res_text)

In [26]:
tokenizator = Tokenizator(counter_list, 10)

In [27]:
df_tokenized = df

In [29]:
df_tokenized = df_tokenized.drop(['forwarded_from'], axis=1)

In [30]:
df_tokenized['tokenized_text'] = df_tokenized['text'].apply(tokenizator.tokenize)

In [32]:
slovarb = {v: k for k,v in enumerate(df_tokenized['from'].unique())}

In [35]:
df_tokenized['tokenized_sender'] = df_tokenized['from'].apply(lambda x: torch.tensor(slovarb[x]))

In [37]:
df_tokenized.to_pickle('data/tokenized_data.pkl')

In [38]:
df1 = pd.read_pickle('data/tokenized_data.pkl')